In [2]:

!pip install pinecone


In [1]:
from pinecone import Pinecone, ServerlessSpec
import os
from openai import OpenAI
import pandas as pd
from time import time
import dotenv
dotenv.load_dotenv()

c:\Users\ASUS\miniconda3\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:

model_name = "meta-llama/Llama-3.2-3B-Instruct"
pinecone_api_key = os.getenv("PINECONE_API_KEY") 

In [ ]:
pc = Pinecone(api_key=pinecone_api_key)

## Try out embeddings

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


c:\Users\ASUS\miniconda3\envs\gpu\lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


## Load model

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True,return_dict_in_generate=True,torch_dtype = torch.float16,device_map='auto')


text = "AI is revolutionizing the world."


Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.82s/it]


## embedding func

In [ ]:
def gen_embe(txt):

    inputs = tokenizer(txt, return_tensors="pt")
    inputs.to("cuda")
    
    with torch.no_grad():
        outputs = model(**inputs)


    hidden_states = outputs.hidden_states  
    last_hidden_state = hidden_states[-1]  

    
    embedding = last_hidden_state.mean(dim=1)  

    return embedding

## Wrangle dataset

In [9]:
df=pd.read_json('products\products.jsonl',lines=True)

In [11]:
df.head(2)

,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,SavoryScone.webp


In [12]:
df['text'] =  df['name']+" : "+df['description'] + \
                " -- Ingredients: " + df['ingredients'].astype(str) + \
                " -- Price: " + df['price'].astype(str) + \
                " -- rating: " + df['rating'].astype(str) 

In [13]:
df['text'].head()

0    Cappuccino : A rich and creamy cappuccino made...
1    Jumbo Savory Scone : Deliciously flaky and but...
2    Latte : Smooth and creamy, our latte combines ...
3    Chocolate Chip Biscotti : Crunchy and delightf...
4    Espresso shot : A bold shot of rich espresso, ...
Name: text, dtype: object

In [14]:
texts = df['text'].tolist()

In [15]:
with open('products\Arjav_shop_about_us.txt') as f:
    Merry_way_about_section = f.read()
    
Merry_way_about_section = "Coffee shop Arjav about section: " + Merry_way_about_section
texts.append(Merry_way_about_section)

In [16]:
with open('products\menu_items_text.txt') as f:
    menue_items_text = f.read()
    
menue_items_text = "Menu Items: " + menue_items_text
texts.append(menue_items_text)

## Push data to database

In [ ]:
index_name = "coffeeshop"

In [ ]:
pc.create_index(
    name=index_name,
    dimension=3072, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '65839af6c17fee3a62c0c1316dbd7df2;o=1', 'date': 'Fri, 28 Mar 2025 19:36:17 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [27]:
embeddings = []
for i in texts:
    emb = gen_embe(i)
    print(len(emb[0]))
    embeddings.append(emb[0])

3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072
3072


In [ ]:

while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

vectors = []

for text, e in zip(texts, embeddings):
    entry_id = text.split(":")[0].strip()
    vectors.append({
        "id": entry_id,
        "values": e,
        "metadata": {'text': text}
    })
    
index.upsert(
    vectors=vectors,
    namespace="ns1"
)

{'upserted_count': 20}

## Get Closest documents

In [12]:
emb = gen_embe("Is Vanilla  -free?")
print(len(emb[0].cpu().numpy()))
emb = emb[0].cpu().tolist()

3072


In [41]:
print(type(emb))

<class 'list'>


In [ ]:
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [13]:
results = index.query(
    namespace="ns1",
    vector=emb,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Sugar Free Vanilla syrup',
              'metadata': {'text': 'Sugar Free Vanilla syrup : Enjoy the sweet '
                                   'flavor of vanilla without the sugar, '
                                   'making it perfect for your coffee or '
                                   'dessert. This syrup offers a guilt-free '
                                   'way to enhance your beverages, ensuring '
                                   'you never miss out on flavor. -- '
                                   "Ingredients: ['Water', 'Natural Flavors', "
                                   "'Sucralose'] -- Price: 1.5 -- rating: 4.4"},
              'score': 0.62770927,
              'values': []},
             {'id': 'Carmel syrup',
              'metadata': {'text': 'Carmel syrup : Sweet and creamy, our '
                                   'caramel syrup is ideal for topping your '
                                   'drinks and desserts with a rich caramel '
  

AttributeError: 'LlamaForCausalLM' object has no attribute 'embeddings'

In [2]:
from pinecone import Pinecone
import os

c:\Users\ASUS\miniconda3\envs\gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

model_name = "meta-llama/Llama-3.2-3B-Instruct"
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)
index_name = 'coffeeshop'

In [6]:
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

index = pc.Index(index_name)

In [11]:
em= [1]*3072

In [12]:
results = index.query(
    namespace="ns1",
    vector=em,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

{'matches': [{'id': 'Menu Items',
              'metadata': {'text': 'Menu Items: Menu Items\n'
                                   '\n'
                                   'Cappuccino - $4.50\n'
                                   'Jumbo Savory Scone - $3.25\n'
                                   'Latte - $4.75\n'
                                   'Chocolate Chip Biscotti - $2.50\n'
                                   'Espresso shot - $2.00\n'
                                   'Hazelnut Biscotti - $2.75\n'
                                   'Chocolate Croissant - $3.75\n'
                                   'Dark chocolate (Drinking Chocolate) - '
                                   '$5.00\n'
                                   'Cranberry Scone - $3.50\n'
                                   'Croissant - $3.25\n'
                                   'Almond Croissant - $4.00\n'
                                   'Ginger Biscotti - $2.50\n'
                                   'Oatmeal Scone - $3.